# Setup

## Step-by-Step Guide for Anaconda Setup

To ensure compatibility and avoid package conflicts, it's recommended to create a new conda environment with a specific Python version. Here are the steps to do this:

### 1. Install Anaconda
First, if you don't already have Conda installed, you'll need to install either Miniconda or Anaconda:

- [Download Anaconda](https://www.anaconda.com/products/distribution)

### 2. Create a New Conda Environment

#### For macOS, Linux, and Windows:
1. Open your terminal or command prompt.
2. Create a new environment with Python 3.8:

```sh
conda create -n FAC python=3.8
```

3.	Activate the new environment:

```sh
conda activate FAC
```

<!-- 4. Install the required python libraries

```sh
pip install -r requirements.txt
``` -->

## Facer

First thing after getting our Anaconda environment ready, we have to ensure that we have the proper models working and running for inference.

##### We install [Facer](https://github.com/FacePerceiver/facer) by using `pip install`

To make it work properly we will need to have git-lfs (Git Large File Storage) installed first

**macOS**

To install Git LFS on macOS, use Homebrew:
```sh
brew install git-lfs
```

**Linux**

To install Git LFS on Linux, use APT:
```sh
sudo apt-get install git-lfs
```

**Windows**

To install Git LFS on Windows, download the installer from the [Git LFS website](https://git-lfs.github.com/).

After installing Git LFS, run the following command in command prompt to initialize it (OS agnostic):
```sh
git lfs install
```

And now we should be ready to go with the needed instalations.

#### Facer installation will handle the installation of the required packages for the facer program

In [ ]:
pip install git+https://github.com/FacePerceiver/facer.git@main

## Warning!

Facer is [CUDA](https://docs.nvidia.com/cuda/doc/index.html) dependent and it means that users without a NVIDIA GPU won't be able to run the code unless you provide changes in facer native file 'classification.py': 
1. at line **62** from `self.cuda().float()` to `self.cpu().float()`
2. at line **141** from `model.cuda()` to `model.cpu()`

## Dataset classification results extraction script (Around 10h long running time)

In [ ]:
import facer
import os
import zipfile
import torch
import pandas as pd

def extract_zip(zip_file):
    img_dir = os.path.splitext(zip_file)[0]
    
    if not os.path.exists(img_dir):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(os.path.dirname(zip_file))
        print(f"Extracted {zip_file}")
    else:
        print(f"{img_dir} already exists. Skipping extraction.")

    return img_dir

device = "cpu"

zip_file = "../img_align_celeba.zip" 
image_dir = extract_zip(zip_file)

face_detector = facer.face_detector("retinaface/mobilenet", device=device)
face_attr = facer.face_attr("farl/celeba/224", device=device)

results = []

images = os.listdir(image_dir)

for img_file in images:
    if img_file.endswith('.jpg'):
        img_path = os.path.join(image_dir, img_file)
        
        # Read and process image
        image = facer.hwc2bchw(facer.read_hwc(img_path)).to(device=device)
        
        with torch.inference_mode():        
            faces = face_detector(image)   
            
            if faces['image_ids'].shape[0] == 0:
                print(f"No faces detected in {img_file}")
                continue
            
            faces = face_attr(image, faces)
        
        labels = face_attr.labels
        
        for i, face_attrs in enumerate(faces["attrs"]):
            face_data = {"image": img_file}
            for prob, label in zip(face_attrs, labels):
                face_data[label] = prob.item()
            results.append(face_data) 
    
df = pd.DataFrame(results)

output_csv = "../celeba_face_attributes.csv"
df.to_csv(output_csv, index=False)

print(f"Saved face attributes to {output_csv}")

## SwinFace

1. Replace `inference.py` file in locally cloned SwinFace with provided `utils\inference.py` and run `python3 inference.py` in the cloned SwinFace repository folder.
2. Move the obtained `results.csv` into a predecessing folder for this repository.
3. Adjust the `results.csv` obtained from running the `inference.py` on CelebA dataset by execting below cell.

In [28]:
import pandas as pd

def reorder_columns(input_file_path, output_file_path):
    df = pd.read_csv(input_file_path)

    # Replace spaces with underscores in column names
    df.columns = [col.replace(' ', '_') for col in df.columns]
    
    # Rename columns with special characters and amibiguities
    df = df.rename(columns={
        'Gender': 'Male',
        "Five_O'Clock_Shadow": "5_o_Clock_Shadow"
    })
    
    new_column_order = [
        "image", "5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs", 
        "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", 
        "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", 
        "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin", 
        "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair", 
        "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", 
        "Wearing_Necktie", "Young"
    ]

    df_reordered = df[new_column_order]
    df_reordered.to_csv(output_file_path, index=False)

input_file = '../result.csv'
output_file = '../result_reordered.csv'
reorder_columns(input_file, output_file)